### Import and initialize function from pyyparksql

In [2]:
from pyspark.sql import SparkSession, Row

### Create SparkSession

In [5]:
spark = SparkSession.Builder().appName("SparkSQL").getOrCreate()

* `SparkSession` is the entry point to programming Spark with the Dataset and DataFrame API. It’s the new entry point that supersedes the old `SparkContext`, `SQLContext`, and `HiveContext`. `SparkSession` provides a unified interface for interacting with Spark.
* The `builder` method is used to create a `SparkSession`.Builder object, which provides various options for configuring and creating a SparkSession. The `builder` method is a static method, so it is called on the SparkSession class itself.
* The `appName` method sets a name for the application. This name will appear in the Spark UI and can be useful for identifying your application. In this case, the application name is set to "SparkSQL". The appName method is called on the SparkSession.Builder object.
* The `getOrCreate` method is used to either get an existing `SparkSession` or, if none exists, create a new one. This is useful for ensuring that there is only one SparkSession per application.

### Defined mapper() function

In [6]:
def mapper(line):
    fields = line.split(',')
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), \
               age=int(fields[2]), numFriends=int(fields[3]))

This function takes a line of text, splits it by `commas`, and returns a `Row` object with fields `ID`, `name`, `age`, and `numFriends`. The Row object helps define the schema for the DataFrame.

### Reading and Mapping Input Data

In [15]:
lines = spark.sparkContext.textFile("./source/fakefriends.csv")
people = lines.map(mapper)

* `spark.sparkContext.textFile("./source/fakefriends.csv")`: Reads the CSV file into an RDD called lines.
* `lines.map(mapper)`: Applies the mapper function to each line, converting it to a Row object. This results in an RDD of Row objects called people.

### Creating DataFrame and Registering as a Table

In [ ]:
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")

* `spark.createDataFrame(people).cache()`: Converts the people RDD to a DataFrame and caches it for faster access.
* `schemaPeople.createOrReplaceTempView("people")`: Registers the DataFrame as a temporary table named "people". This allows SQL queries to be run against it.

### Running SQL Queries

In [ ]:
# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are RDDs and support all the normal RDD operations.
for teen in teenagers.collect():
  print(teen)

* `spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")`: Executes an SQL query to select rows where age is between 13 and 19, inclusive. The result is a DataFrame named teenagers.
* `teenagers.collect()`: Collects the results of the query back to the driver program as a list of rows.

### Performing data operation

In [ ]:
# We can also use functions instead of SQL queries:
schemaPeople.groupBy("age").count().orderBy("age").show()

* `schemaPeople.groupBy("age").count()`: Groups the DataFrame by the age column and counts the number of rows for each age.
* `orderBy("age").show()`: Orders the results by the age column and displays them.

In [ ]:
spark.stop()
# Stop the Spark session to release resourcess.